In [1]:
from bs4 import BeautifulSoup
from requests import get
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [2]:
headers = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})

In [11]:
sapo = "https://www.bezrealitky.cz/nemovitosti-byty-domy/539200-nabidka-prodej-bytu-tetinska-hlavni-mesto-praha"
response = get(sapo, headers=headers)


In [12]:
print(response.text)

<!DOCTYPE html>
<html lang="cs" class="no-js" prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb#">
    <head>
        <script>
                window.dataLayer = window.dataLayer || [];
            </script><script>
        dataLayer.push({"pageType":"inzerat_prodej","equipped":"\u010c\u00e1ste\u010dn\u011b","price":7495000,"currency":"CZK","status":"active","geo1":"Praha","geo2":"Radlice"});
    </script><script>
                var userId = '';
                if (userId !== '') {
                    dataLayer.push({'userID': userId});
                }
            </script><!-- Google Tag Manager --><script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
                new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
                j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
                'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
                })(window,document,'script',

In [6]:
html_soup = BeautifulSoup(response.text, 'html.parser')

In [7]:
house_containers = html_soup.find_all('span', class_="items__item")

In [8]:
print(house_containers)

[]


In [7]:
first = house_containers[0]
first.find_all('span')

[<span class="messengerOFFLINE" id="MC_PropertyInList_repProperties_spanMessenger_0" onclick="goToMessenger('/0d21bf8e-766e-11e9-9763-060000000053.html');" style="cursor: pointer;">
 <i class="fa fa-comment fa-lg"></i>
 </span>, <span>
                         Apartamento T3, Bairro das Colónias (Anjos), Arroios, Lisboa
                     </span>, <span class="btnContactPVPI" id="MC_PropertyInList_repProperties_btnContactPVPINormal_0" onclick="ShowContactForm('0d21bf8e-766e-11e9-9763-060000000053', '13', '5', true); return false;" style="z-index: 9999;">Contacte Anunciante</span>, <span>310 000 <strong title="Euro">€</strong></span>]

In [8]:
titles = []
created = []
prices = []
areas = []
zone = []
condition = []
descriptions = []
urls = []
thumbnails = []

In [ ]:
%%time

n_pages = 0

for page in range(0,900):
    n_pages += 1
    sapo_url = 'https://casa.sapo.pt/Venda/Apartamentos/?sa=11&lp=10000&or=10'+'&pn='+str(page)
    r = get(sapo_url, headers=headers)
    page_html = BeautifulSoup(r.text, 'html.parser')
    house_containers = page_html.find_all('div', class_="searchResultProperty")
    if house_containers != []:
        for container in house_containers:
            
            # Price            
            price = container.find_all('span')[2].text
            if price == 'Contacte Anunciante':
                price = container.find_all('span')[3].text
                if price.find('/') != -1:
                    price = price[0:price.find('/')-1]
            if price.find('/') != -1:
                price = price[0:price.find('/')-1]
            
            price_ = [int(price[s]) for s in range(0,len(price)) if price[s].isdigit()]
            price = ''
            for x in price_:
                price = price+str(x)
            prices.append(int(price))

            # Zone
            location = container.find_all('p', class_="searchPropertyLocation")[0].text
            location = location[7:location.find(',')]
            zone.append(location)

            # Title
            name = container.find_all('span')[0].text
            titles.append(name)

            # Status
            status = container.find_all('p')[5].text
            condition.append(status)

            # Area
            m2 = container.find_all('p')[9].text
            if m2 != '-':
                m2 = m2.replace('\xa0','')
                m2 = float("".join(itertools.takewhile(str.isdigit, m2)))
                areas.append(m2)
                
            else:
                m2 = container.find_all('p')[7].text
                if m2 != '-':
                    m2 = m2.replace('\xa0','')
                    m2 = float("".join(itertools.takewhile(str.isdigit, m2)))
                    areas.append(m2)
                else:
                    areas.append(m2)

            # Creation date
            date = pd.to_datetime(container.find_all('div', class_="searchPropertyDate")[0].text[21:31])
            created.append(date)

            # Description
            desc = container.find_all('p', class_="searchPropertyDescription")[0].text[7:-6]
            descriptions.append(desc)

            # url
            link = 'https://casa.sapo.pt/' + container.find_all('a')[0].get('href')[1:-6]
            urls.append(link)

            # image
            img = str(container.find_all('img')[0])
            img = img[img.find('data-original_2x=')+18:img.find('id=')-2]
            thumbnails.append(img)
    else:
        break
    
    sleep(randint(1,2))
    
print('You scraped {} pages containing {} properties.'.format(n_pages, len(titles)))